In [9]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string

In [10]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [11]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [12]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [13]:
fake['class'] = 0
true['class'] = 1

In [14]:
data = pd.concat([fake,true],axis = 0)

In [15]:
data.sample(10)

,title,text,subject,date,class
23093,SUNDAY SCREENING: ‘A Noble Lie’ (2011),21st Century Wire says Our weekly documentary ...,Middle-east,"December 18, 2016",0
3402,Deutsche Bank ignores U.S. Trump/Russia query:...,FRANKFURT (Reuters) - Germany’s largest bank h...,politicsNews,"June 4, 2017",1
8977,Celebrity Gun Nut Threatens To Shoot His Girl...,A man known primarily for posting videos of hi...,News,"January 4, 2016",0
14134,"Czech parliament picks speaker, opens way for ...",PRAGUE (Reuters) - The newly-elected Czech low...,worldnews,"November 22, 2017",1
2645,"Despite recusal, Trump has confidence in Sessi...",WASHINGTON (Reuters) - President Donald Trump ...,politicsNews,"July 20, 2017",1
4503,Early Voting In North Carolina Shows GREAT NE...,Early voting has begun in the battleground sta...,News,"September 23, 2016",0
3675,New Russia probe leaks threaten to derail Trum...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"May 19, 2017",1
15731,LYING WHITE HOUSE PRESS SECRETARY: “OBAMA HAS ...,It s interesting that Josh Earnest still carri...,politics,"May 7, 2015",0
14238,GEORGE WASHINGTON PROFESSOR On Soros Activists...,Americans better start fighting back against t...,politics,"Mar 21, 2016",0
15925,Syrian opposition rejects Russia-sponsored pea...,BEIRUT/AMMAN/ANKARA (Reuters) - Syria s opposi...,worldnews,"November 1, 2017",1


In [16]:
data = data.drop(['title','subject','date'],axis = 1)

In [17]:
data.reset_index(inplace = True)

In [18]:
data.drop(["index"],axis = 1,inplace = True)

In [19]:
data.sample(5)

,text,class
41050,OSLO (Reuters) - Norway said on Friday it plan...,1
25218,WASHINGTON (Reuters) - The U.S. Justice Depart...,1
33796,WASHINGTON (Reuters) - South Carolina Governor...,1
29613,(Reuters) - An order by President Donald Trump...,1
35795,VIENNA (Reuters) - Tuesday s blast at Austria ...,1


In [20]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)  # Remove content in brackets
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)  # Remove punctuation
    text = re.sub(r'\n', ' ', text)  # Replace newlines with space
    text = re.sub(r'\w*\d\w*', '', text)  # Remove words with digits
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

In [21]:
print(data["text"].apply(type).value_counts())

text
<class 'str'>    44898
Name: count, dtype: int64


In [22]:
data["text"] = data["text"].astype(str)

In [23]:
data["text"] = data["text"].apply(clean_text)

In [24]:
x = data["text"]
y = data["class"]

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.25,random_state=42)

In [25]:
print(type(xtrain), type(xtrain.iloc[0]))

<class 'pandas.core.series.Series'> <class 'str'>


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

xtrain = xtrain.tolist()
xtest = xtest.tolist()

vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [27]:
print(xv_train.shape) 
print(type(xv_train))

(33673, 173161)
<class 'scipy.sparse._csr.csr_matrix'>


In [28]:
lr = LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

In [29]:
prediction = lr.predict(xv_test)
lr.score(xv_test,ytest)

0.985924276169265

In [30]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [31]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,'lr_model.jb')

['lr_model.jb']